In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [27]:
# read data from input files
test_dataset = pd.read_csv("/kaggle/input/titanic/test.csv")
train_dataset = pd.read_csv("/kaggle/input/titanic/train.csv")

In [28]:
print(set(train_dataset['Sex']))
set(train_dataset['Pclass'])
set(train_dataset['Parch'])
set(train_dataset['Cabin'])
set(train_dataset['Embarked'])

{'male', 'female'}


{'C', 'Q', 'S', nan}

In [29]:
train_dataset.isnull().sum()
train_dataset.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [30]:
#remove missing values from Age

train_dataset = train_dataset[['Pclass','Sex','Age','Parch','Survived']]
train_dataset.dropna(axis=0,inplace=True)
train_dataset.count()

Pclass      714
Sex         714
Age         714
Parch       714
Survived    714
dtype: int64

In [31]:
#covert catergorical variable Sex to numerical
train_dataset.replace(['male','female'],[0,1],inplace=True)
train_dataset.head()

,Pclass,Sex,Age,Parch,Survived
0,3,0,22.0,0,0
1,1,1,38.0,0,1
2,3,1,26.0,0,1
3,1,1,35.0,0,1
4,3,0,35.0,0,0


In [32]:
#save the dependent variable
trainX = train_dataset[['Pclass','Sex','Age','Parch']].values
trainY = train_dataset[['Survived']].values

trainY[:5],trainX[:5]

(array([[0],
        [1],
        [1],
        [1],
        [0]]),
 array([[ 3.,  0., 22.,  0.],
        [ 1.,  1., 38.,  0.],
        [ 3.,  1., 26.,  0.],
        [ 1.,  1., 35.,  0.],
        [ 3.,  0., 35.,  0.]]))

In [33]:
#fitting the model using decision tree
from sklearn.tree import DecisionTreeClassifier
des_tree = DecisionTreeClassifier(criterion='entropy',max_depth=10)
des_tree.fit(trainX,trainY)


DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [34]:
#test data
test_dataset = pd.read_csv("/kaggle/input/titanic/test.csv")
test_dataset.count()

#replace missing values from Age with mean

test_dataset1 = test_dataset[['Pclass','Sex','Age','Parch']]
test_dataset1['Age'].fillna(test_dataset1['Age'].mean(),inplace=True)
test_dataset1.count()

#covert catergorical variable Sex to numerical
test_dataset1.replace(['male','female'],[0,1],inplace=True)
test_dataset1.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Pclass,Sex,Age,Parch
0,3,0,34.5,0
1,3,1,47.0,0
2,2,0,62.0,0
3,3,0,27.0,0
4,3,1,22.0,1


In [35]:
testX = test_dataset1[['Pclass','Sex','Age','Parch']].values
testX

array([[ 3.        ,  0.        , 34.5       ,  0.        ],
       [ 3.        ,  1.        , 47.        ,  0.        ],
       [ 2.        ,  0.        , 62.        ,  0.        ],
       ...,
       [ 3.        ,  0.        , 38.5       ,  0.        ],
       [ 3.        ,  0.        , 30.27259036,  0.        ],
       [ 3.        ,  0.        , 30.27259036,  1.        ]])

In [36]:
predictY = des_tree.predict(testX)
len(predictY)

output = test_dataset[['PassengerId']]
output['Survived'] = predictY
output.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1


In [37]:
#convert pandas to excel
filename = "./outputs/output_titanic.csv"
output.to_csv(filename,index=False)